# Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

import re
from sklearn.model_selection import train_test_split

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Reading the Dataset

In [4]:
dataset = pd.read_csv('Sentiment.csv')

In [5]:
dataset

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0000,yes,1.0000,Neutral,0.6578,None of the above,1.0000,NaN,...,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0000,yes,1.0000,Positive,0.6333,None of the above,1.0000,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0000,yes,1.0000,Neutral,0.6629,None of the above,0.6629,NaN,...,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0000,yes,1.0000,Positive,1.0000,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0000,yes,1.0000,Positive,0.7045,None of the above,1.0000,NaN,...,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13866,13867,No candidate mentioned,1.0000,yes,1.0000,Negative,0.7991,Abortion,0.6014,No candidate mentioned,...,yes,7,Negative,Abortion\nWomen's Issues (not abortion though),RT @cappy_yarbrough: Love to see men who will ...,NaN,2015-08-07 09:29:43 -0700,629690895479250944,Como,NaN
13867,13868,Mike Huckabee,0.9611,yes,1.0000,Positive,0.7302,None of the above,0.9229,Mike Huckabee,...,yes,1,NaN,NaN,RT @georgehenryw: Who thought Huckabee exceede...,NaN,2015-08-07 09:25:02 -0700,629689719056568320,USA,NaN
13868,13869,Ted Cruz,1.0000,yes,1.0000,Positive,0.8051,None of the above,0.9647,Ted Cruz,...,yes,67,Positive\nNeutral,NaN,"RT @Lrihendry: #TedCruz As President, I will a...",NaN,2015-08-07 07:19:18 -0700,629658075784282112,NaN,NaN
13869,13870,Donald Trump,1.0000,yes,1.0000,Negative,1.0000,Women's Issues (not abortion though),0.9202,Donald Trump,...,yes,149,NaN,Women's Issues (not abortion though),RT @JRehling: #GOPDebate Donald Trump says tha...,NaN,2015-08-07 09:54:04 -0700,629697023663546368,NaN,NaN


In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13871 entries, 0 to 13870
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         13871 non-null  int64  
 1   candidate                  13775 non-null  object 
 2   candidate_confidence       13871 non-null  float64
 3   relevant_yn                13871 non-null  object 
 4   relevant_yn_confidence     13871 non-null  float64
 5   sentiment                  13871 non-null  object 
 6   sentiment_confidence       13871 non-null  float64
 7   subject_matter             13545 non-null  object 
 8   subject_matter_confidence  13871 non-null  float64
 9   candidate_gold             28 non-null     object 
 10  name                       13871 non-null  object 
 11  relevant_yn_gold           32 non-null     object 
 12  retweet_count              13871 non-null  int64  
 13  sentiment_gold             15 non-null     obj

# extract text and sentiment column from dataset


In [7]:
dataset = dataset[['text','sentiment']]

In [8]:
dataset

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive
...,...,...
13866,RT @cappy_yarbrough: Love to see men who will ...,Negative
13867,RT @georgehenryw: Who thought Huckabee exceede...,Positive
13868,"RT @Lrihendry: #TedCruz As President, I will a...",Positive
13869,RT @JRehling: #GOPDebate Donald Trump says tha...,Negative


 **remove row from dataset where sentiment value equal neutral. classify tweets into positive and**

In [9]:
dataset = dataset[dataset['sentiment']!= 'Neutral']

In [10]:
dataset

,text,sentiment
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive
5,"RT @GregAbbott_TX: @TedCruz: ""On my first day ...",Positive
6,RT @warriorwoman91: I liked her and was happy ...,Negative
...,...,...
13866,RT @cappy_yarbrough: Love to see men who will ...,Negative
13867,RT @georgehenryw: Who thought Huckabee exceede...,Positive
13868,"RT @Lrihendry: #TedCruz As President, I will a...",Positive
13869,RT @JRehling: #GOPDebate Donald Trump says tha...,Negative


## Data Preprocessing

In [11]:
dataset['text'][9720]

'RT @RWSurferGirl: It is very disappointing that Fox News is not conducting a "Fair &amp; Balanced" Debate. 🇺🇸 #GOPDebate  #GOPDebates'

**remove @ word # word and RT word which is not having any sentiment **

In [12]:
pattern = "(#\w+)|(RT\s@\w+:)|(http.*)|(@\w+)"

In [13]:
ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [14]:
def Clean_text(data):
    tweets = []
    sentiments = []
    for index,row in data.iterrows():
        sentence = re.sub(pattern,'',row.text)
        words = [e.lower() for e in sentence.split()]
        words = [lemmatizer.lemmatize(word) for word in words if word not in stopwords.words('english')]
        words = ' '.join(words)
        tweets.append(words)
        sentiments.append(row.sentiment)
    return tweets,sentiments

In [15]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [16]:
dataset_tweets,dataset_sentiments = Clean_text(dataset)

In [17]:
dataset_tweets

["catch full last night. scott's best line 90 seconds.",
 'carly fiorina trending -- hour debate -- men just-completed say …',
 'w/ delivered highest rating history presidential debates.',
 ': "on first day rescind every illegal executive action taken barack obama."',
 'liked happy heard going moderator. anymore.',
 'deer headlight ben carson, may brain surgeon performed lobotomy himself.',
 "last night's debate proved it!",
 'fairness owns phrase.',
 'woke tweet best line night via',
 "reading family's comment great",
 "loved scott walker mark harmon's romantic rival summer school. look up.",
 "hey exploiting tragedy 9/11 political gain 's thing",
 'fire comment woman @ weigh',
 'guess speaking time . moderator 31.7% time.',
 "reason comment funny 'in case ignorant' reason government working people",
 'huckabee: paying transgender surgery soldiers, sailor airman make country safer',
 'cruz class &amp; truth, get vote!',
 "rt “: still clown show! i'm glad head clown trump helping destr

**combine both column to make dataset frame **

In [18]:
final_data = {'tweets':dataset_tweets,'sentiments':dataset_sentiments}

In [19]:
processed_data = pd.DataFrame(final_data)

In [20]:
processed_data

,tweets,sentiments
0,catch full last night. scott's best line 90 se...,Positive
1,carly fiorina trending -- hour debate -- men j...,Positive
2,w/ delivered highest rating history presidenti...,Positive
3,": ""on first day rescind every illegal executiv...",Positive
4,liked happy heard going moderator. anymore.,Negative
...,...,...
10724,love see men never faced pregnancy talk body ❤...,Negative
10725,thought huckabee exceeded expectation,Positive
10726,"president, always tell truth, said would do.",Positive
10727,donald trump say time political correctness. c...,Negative


**split dataset 10 percent for testing and 90 percent for training as test size=0.1 **

In [21]:
train_data,test_data = train_test_split(processed_data,test_size=0.1,random_state=42)

In [22]:
test_data

,tweets,sentiments
592,done last night showed true color shined!,Positive
608,megyn officially charted moonbat hit parade.,Positive
424,great theatrical performance. broadway.,Negative
10134,hugged obama deserve stage tonight. 🇺🇸,Negative
5635,retweet tired democrat importing third world p...,Positive
...,...,...
1562,"hate strong, independent, women. i'm shocked t...",Negative
1511,wrong! attacked start. nothing fair balanced. ht…,Negative
7134,well blast. strong field . going fun campaign ...,Positive
2523,one complain america's corrupt 2 party system ...,Positive


In [23]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()


In [24]:
train_data['sentiments'] = labelencoder.fit_transform(train_data['sentiments'])

In [25]:
train_data

,tweets,sentiments
9159,great - let's bunch rich entitled men make dec...,0
7229,fave. . ambushed. changed worse,0
3070,included show hand like 1st day school. we're ...,0
1923,"like driving fellow patient around ""one flew c...",0
7377,candidate received word god?! presidential hop...,0
...,...,...
5734,trump's drunk guy one want party,0
5191,mind john kasich next president!,1
5390,"looking ""sift bulls***,"" jon stewart put well:",0
860,stop stoking dishonesty. head gotten big due q...,0


In [26]:
test_data['sentiments'] = labelencoder.fit_transform(test_data['sentiments'])

In [27]:
test_data

,tweets,sentiments
592,done last night showed true color shined!,1
608,megyn officially charted moonbat hit parade.,1
424,great theatrical performance. broadway.,0
10134,hugged obama deserve stage tonight. 🇺🇸,0
5635,retweet tired democrat importing third world p...,1
...,...,...
1562,"hate strong, independent, women. i'm shocked t...",0
1511,wrong! attacked start. nothing fair balanced. ht…,0
7134,well blast. strong field . going fun campaign ...,1
2523,one complain america's corrupt 2 party system ...,1



## Converting Words into Vectors

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(1,3))
cv.fit(train_data['tweets'])

CountVectorizer(ngram_range=(1, 3))

In [29]:
X_train = cv.transform(train_data['tweets'])

In [30]:
print(X_train.shape)

(9656, 82010)


In [31]:
X_train.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [32]:
y_train= train_data['sentiments'].values

In [33]:
y_train

array([0, 0, 0, ..., 0, 0, 0])

In [34]:
X_test = cv.transform(test_data['tweets'])

In [35]:
X_test.shape

(1073, 82010)

In [36]:
X_test.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [37]:
y_test= test_data['sentiments'].values

In [38]:
y_test


array([1, 1, 0, ..., 1, 1, 0])

## Sentiment Analysis (Model Building)

In [39]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()

In [42]:
classifier.fit(X_train,y_train)


MultinomialNB()

In [43]:
y_pred = classifier.predict(X_test)

In [44]:
y_pred

array([0, 0, 1, ..., 0, 0, 0])

# accuracy Evaluation

In [45]:
from sklearn.metrics import accuracy_score

acc=accuracy_score(y_pred,y_test)
print(acc)
print("test accuracy: {:.2f}%".format(acc*100))

0.8462255358807083
test accuracy: 84.62%


# predict output

In [46]:
classifier.predict(X_test[[0]])

array([0])

In [47]:
from sklearn.metrics import  classification_report, confusion_matrix, ConfusionMatrixDisplay
import warnings

# Model evaluation:

In [48]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Evaluate the model on the test data

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Accuracy: 0.8462255358807083
Precision: 0.8005290311297657
Recall: 0.6813162753890794
F1-score: 0.7137181471114711


# Exporting the model:

In [49]:
import joblib

# Save the model to disk
filename = 'tweets_sentiment_analyser_model.joblib'
joblib.dump(classifier, filename)

# Load the model from disk
loaded_model = joblib.load(filename)